In [2]:
from keras.datasets import mnist

#Load Dataset
(x_train , y_train) ,(x_test , y_test) = mnist.load_data()

print(x_train.shape) #printing shape of training images , 60,000 images
print(x_test.shape) #printing shape of test images , 10,000 images

Using TensorFlow backend.


(60000, 28, 28)
(10000, 28, 28)


In [3]:
#mnist dataset are 28*28 images with no other color channels
print("Samples in train data" + str(x_train.shape))
print("Samples in test data" + str(x_test.shape))

print("Dimensions of train data" + str(x_train[0].shape))
print("Dimensions of test data" + str(x_test[0].shape))

print("Labels in train data" + str(y_train.shape))
print("Labels in test data" + str(y_test.shape))

Samples in train data(60000, 28, 28)
Samples in test data(10000, 28, 28)
Dimensions of train data(28, 28)
Dimensions of test data(28, 28)
Labels in train data(60000,)
Labels in test data(10000,)


In [4]:
#displaying images using opencv or matplotlib
import cv2
import numpy as np
import matplotlib.pyplot as plt


for i in range(0,6):
    random_num = np.random.randint(0 , len(x_train)) #generates a number from 0 to 60000
    img = x_train[random_num]
    window_name = "Random Sample" + str(i)
    cv2.imshow("ss",img)
    cv2.waitKey()
    
    
cv2.destroyAllWindows()

#plotting using matplotlib
for i in range(331 , 335):
    plt.subplot(i)
    random_num = np.random.randint(0 , len(x_train))
    plt.imshow(x_train[random_num] ,cmap = plt.get_cmap('gray'))
    
plt.show()

<Figure size 640x480 with 4 Axes>

In [5]:
#converting data into required shape
#keras requires in shape = (Number of Samples , Rows , Columns , Depth)
#in four dimensions , currently our data is (60000,28,28,1) in color (60000,28,28,3)
#using numpys reshape we can add the extra dimension
img_rows = x_train[0].shape[0] #takes the first image and the rows
img_cols = x_train[0].shape[1] #takes the first image and returns columns
print(img_rows)
print(img_cols)

x_train = x_train.reshape(x_train.shape[0] , img_rows , img_cols , 1)
x_test = x_test.reshape(x_test.shape[0] , img_rows , img_cols , 1)
#x_train.shape[0] returns the number of samples that is 60000


#store and declare the input shape for the first layer of the cnn
input_shape = (img_rows , img_cols , 1)

#keras expects float32 datatype
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#Normalize data to change the range from 0-255 to 0-1
x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

28
28
(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [6]:
#one hot encoding the labels because keras can only process numerical y data
# one hot enconding represents each label in numerical form where the number of columns is the number of classes
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1]
num_samples = y_test.shape[0]



In [18]:
#Creating our model
#start dropout small and end with 0.5

import keras
from keras.models import Sequential
from keras.layers import Dense , Dropout , Flatten
from keras.layers import Conv2D , MaxPooling2D
from keras import backend as K
from keras.optimizers import SGD

model = Sequential()

model.add(Conv2D(32 , kernel_size = (3,3) ,activation ='relu' , input_shape = input_shape))
model.add(Conv2D(64 , (3,3) , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(120 , activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes , activation='softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = SGD(0.01) , metrics =['accuracy'])

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               1106040   
_________________________________________________________________
dropout_4 (Dropout)          (None, 120)              

In [19]:
#Training  our model
#batch_size  how many samples in each batch depends on ram
batch_size = 32
epochs = 20

final_model = model.fit(x_train , y_train , batch_size = batch_size , epochs = epochs , verbose = 1 , validation_data = (x_test , y_test) )#verbose how much information to see when training

score = model.evaluate(x_test , y_test , verbose = 1)
print("Test loss:" , score[0])
print("Test accuracy:" , score[1])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 219s 4ms/step - loss: 0.5636 - acc: 0.8230 - val_loss: 0.1916 - val_acc: 0.9431
Epoch 2/20
60000/60000 [==============================] - 219s 4ms/step - loss: 0.2972 - acc: 0.9113 - val_loss: 0.1436 - val_acc: 0.9570
Epoch 3/20
60000/60000 [==============================] - 218s 4ms/step - loss: 0.2335 - acc: 0.9305 - val_loss: 0.1161 - val_acc: 0.9645
Epoch 4/20
60000/60000 [==============================] - 219s 4ms/step - loss: 0.1888 - acc: 0.9439 - val_loss: 0.0870 - val_acc: 0.9736
Epoch 5/20
60000/60000 [==============================] - 220s 4ms/step - loss: 0.1520 - acc: 0.9550 - val_loss: 0.0701 - val_acc: 0.9780
Epoch 6/20
60000/60000 [==============================] - 221s 4ms/step - loss: 0.1284 - acc: 0.9621 - val_loss: 0.0576 - val_acc: 0.9814
Epoch 7/20
60000/60000 [==============================] - 223s 4ms/step - 

In [25]:


from keras.models import model_from_json 
from keras.models import load_model

#saving model weights
model.save('C:\\Users\\Zahid\\Desktop\\Deep Learning\\mnist_model_weights.h5')
print("Model Saved")

#save model
model_json = model.to_json()
with open(r'C:\Users\Zahid\Desktop\Deep Learning\mnist_model.json', 'w') as json_file:
    json_file.write(model_json)


classifier = load_model('C:\\Users\\Zahid\\Desktop\\Deep Learning\\mnist_model_weights.h5')

with open(r'C:\Users\Zahid\Desktop\Deep Learning\mnist_model.json','r') as jfile:
            model = model_from_json(jfile.read())
          


Model Saved


C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:303: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [27]:
          #plotting loss and accuracy charts

import matplotlib.pyplot as plt

history_dict = final_model.history

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1 , len(loss_values) + 1)

line1 = plt.plot(epochs , val_loss_values , label = "Validation/Test Loss")
line2 = plt.plot(epochs , loss_values , label = "Training Loss")
plt.setp(line1 , linewidth = 2.0 , marker = '+' , markersize = 10.0)
plt.setp(line2 , linewidth = 2.0 , marker = '4' , markersize = 10.0)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.show()


TypeError: 'History' object is not subscriptable

In [28]:
#Testing Data

def draw_test(name , pred , input_img):
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(input_img , 0,0,0 , imageL.shape[0] , cv2.BORDER_CONSTANT , value = BLACK)
    expanded_image = cv2.cvtColor(expanded_image , cv2.COLOR_GRAY2BGR)
    cv2.putText(expanded_image , str(pred) , (152 , 78) , cv2.FONT_HERSHEY_COMPLEX_SMALL , 4 , (0,255,0) , 2)
    cv2.imshow(name , expanded_image)
    
for i in range(0,10):
    rand = np.random.randint(0,len(x_test))
    input_img = x_test[rand]

    imageL = cv2.resize(input_img , None , fx=4 ,fy=4 , interpolation = cv2.INTER_CUBIC)
    input_img = input_img.reshape(1,28,28,1)

    res = str(classifier.predict_classes(input_img , 1 , verbose = 0)[0])

    draw_test("imagefwe" , res , imageL)
    cv2.waitKey()
    
cv2.destroyAllWindows()

error: OpenCV(4.0.0) C:\projects\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1230: error: (-215:Assertion failed) dst.data == (uchar*)dst_ptr in function 'cvShowImage'


In [29]:
#Visualizing the model
from keras.utils.vis_utils import plot_model
from matplotlib.image import mpimg

model_diagram_path = 'C:\\Users\\Zahid\\Desktop\\Deep Learning'

#generate the model
plot_model(model , to_file = model_diagram_path +'model_plot.png' , show_shapes = True , show_layer_names = True)

img = mpimg.imread(model_diagrams_path + 'model_plot.png')
plt.figure(figsize=(30,15))
imgplot = plt.imshow(img)


ImportError: cannot import name 'mpimg' from 'matplotlib.image' (C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\image.py)